In [1]:
from PIL import Image, ImageOps, ImageDraw, ImageFont, ImageEnhance
import sklearn
import colormath
import numpy as np
import pandas as pd
import colorsys
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
from collections import Counter
import subprocess
import os 
import os.path
from pathlib import Path
import math
import cv2

In [2]:
def minimumOf(valueA, valueB):
    """return minimum, handle type errors"""
    try:
        return min(valueA, valueB)
    except TypeError:
        return f"Error: comparison between {type(valueA)} and {type(valueB)} invalid"
    except Exception as E:
            return f"Unexpected error: {str(e)}"

In [3]:
MAX_WIDTH = 150
MAX_HEIGHT = 200
GRID_SIZE = (1, 1)
MAX_DIMENSION = minimumOf(MAX_WIDTH, MAX_HEIGHT)

In [4]:
def sizingToRatio(original_width, original_height):
    aspect_ratio = original_width / original_height

    if aspect_ratio > 1:
        target_width = minimumOf(MAX_WIDTH, original_width)
        target_height = int(target_width / aspect_ratio)
    elif aspect_ratio < 1: 
        target_height = minimumOf(MAX_HEIGHT, original_height)
        target_width = int(target_height * aspect_ratio)
    else:
        target_width = minimumOf(MAX_WIDTH, original_width)
        target_height = minimumOf(MAX_HEIGHT, original_height)

        target_width = minimumOf(MAX_DIMENSION, original_width)
        target_height = target_width

    return target_width, target_height

In [5]:
def load_and_resize_image(image):
    image.load()
    original_width, original_height = image.size
    target_width, target_height = sizingToRatio(original_width, original_height)

    adjusted_size = (
        (target_width // GRID_SIZE[0]) * GRID_SIZE[0],
        (target_height // GRID_SIZE[1]) * GRID_SIZE[1],
    )
    
    resized_image = image.resize(adjusted_size, Image.Resampling.LANCZOS)
    return resized_image

In [6]:
image_path = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-2.jpeg"
with Image.open(image_path) as image:
    print(f"original size: {image.size}")
    resized_image = load_and_resize_image(image)
    print(f"resized size: {resized_image.size}")

original size: (225, 225)
resized size: (150, 150)


In [7]:
def get_number_of_pixels(image):    
    width, height = image.size
    print(f"w: {width}, h: {height}")
    total_pixels = width * height
    print(f"t: {total_pixels}")
    return width, height, total_pixels

In [8]:
resized_width, resized_height, resized_pixels = get_number_of_pixels(resized_image)
print(f"width: {resized_width}, height: {resized_height}, total_ = {resized_pixels}")

w: 150, h: 150
t: 22500
width: 150, height: 150, total_ = 22500


In [9]:
def quantize_image(image, num_colors):
    """Uses K-Means clustering ML algorithm to reduce number of colors."""
    image = image.convert("RGB")
    image_np = np.array(image)
    pixels = image_np.reshape(-1, 3)

    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=num_colors, n_init=10).fit(pixels)
    palette = kmeans.cluster_centers_.astype(int)  # Quantized color palette
    labels = kmeans.labels_  # Labels for each pixel

    # Reshape the quantized image using the palette and labels
    q_image_np = palette[labels].reshape(image_np.shape)
    q_image = Image.fromarray(q_image_np.astype('uint8'), 'RGB')

    # Convert palette to a list of tuples (each representing an RGB color)
    palette_list = [tuple(map(int, color)) for color in palette]

    return q_image, labels, palette_list

In [10]:
num_colors = 12
quantized_image, labels, palette = quantize_image(image, num_colors)

In [11]:
def load_threads(csv_file):
    colors_df = pd.read_csv(csv_file)
    dmc_colors = {}

    for index, row in colors_df.iterrows():
        color_name = row['ColorName']
        floss_num = row['FlossCode']
        rgb_value = (row['Red'], row['Green'], row['Blue'])
        code_color = f"{floss_num}: {color_name}"
        dmc_colors[code_color] = rgb_value
    
    return dmc_colors

In [12]:
def get_unique_color_count(image):
    image = image.convert('RGB')
    image_np = np.array(image)
    pixels = image_np.reshape(-1, 3)

    unique_colors = np.unique(pixels, axis=0)
    return len(unique_colors)

In [13]:
count = get_unique_color_count(image)
dmc_colors = load_threads('/Users/kateportalatin/py_workspace/stitch_companion_2/dmc_colors.csv')
print(f"image unique colors: {count}")

image unique colors: 14120


In [14]:
def closest_color(target_rgb, color_library):
    # Ensure target_rgb is an iterable and convert it to a tuple of 3 values (R, G, B)
    if isinstance(target_rgb, (np.int32, int)):
        raise TypeError(f"Expected a tuple or list for target_rgb, but got: {target_rgb}")
    
    target_rgb = tuple(map(int, target_rgb))  # Ensure that each value in target_rgb is an integer

    # Convert the RGB color to the sRGBColor object
    target_color = sRGBColor(*target_rgb, is_upscaled=True)

    # Convert the target sRGB color to Lab color space
    target_lab = convert_color(target_color, LabColor)

    closest_color = None
    min_delta_e = float('inf')

    # Iterate over the color library to find the closest match
    for color_name, color_rgb in color_library.items():
        library_color = sRGBColor(*color_rgb, is_upscaled=True)
        library_lab = convert_color(library_color, LabColor)

        # Calculate the color difference using the delta E formula
        delta_e = delta_e_cie2000(target_lab, library_lab)

        if delta_e < min_delta_e:
            min_delta_e = delta_e
            closest_color = (color_name, color_rgb)

    return closest_color


In [15]:
target_color = (70, 130, 180)  # Example: Steel Blue
closest_thread = closest_color(target_color, dmc_colors)
print(f"Closest thread color: {closest_thread}")

Closest thread color: ('825: Blue Dark', (71, 129, 165))


In [16]:
def map_colors_to_thread(palette, color_library):
    mapped_palette = []
    for color in palette: 
        closest_ = closest_color(color, color_library)
        mapped_palette.append({
            'thread_name': closest_[0],
            'rgb_value': closest_[1]
        })                                 
    return mapped_palette

In [17]:
dmc_colors = load_threads('/Users/kateportalatin/py_workspace/stitch_companion_2/dmc_colors.csv')
palette_colors = map_colors_to_thread(palette, dmc_colors)

In [18]:
def find_palette_match(original_color, mapped_palette):
    closest_match = None
    min_distance = float('inf')

    for entry in mapped_palette:
        thread_rgb = entry['rgb_value']

        # Calculate euclidean distance in RGB space
        distance = sum((c1-c2) ** 2 for c1, c2 in zip(original_color, thread_rgb)) ** 0.5

        if distance < min_distance:
            min_distance = distance
            closest_match = entry

    return closest_match

In [43]:
def symbol_dict_create(dmc_colors, mapped_palette):
    symbol_map = {}
    pattern_floss = {}  # dmc colors used in pattern
    symbols = '~@#$%^*-+<>/abcdefghijklmnopqrstuvwxyz1234567890';
    
    for idx, entry in enumerate(mapped_palette):
        thread_name = entry['thread_name']
        thread_rgb = entry['rgb_value']
        
        
        symbol_map[thread_rgb] = symbols[idx]
        pattern_floss[thread_name] = thread_rgb
        
    return symbol_map, pattern_floss

In [44]:
def save_image(image, image_path):
    """ Works with os file system to save output files"""
    directory = os.path.dirname(image_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)
    image.save(image_path)

In [45]:
def apply_palette_to_image(image, labels, mapped_palette, filename='output_image.png'):
    image_np = np.array(image)
    height, width, _ = image_np.shape
    pixels = image_np.reshape(-1, 3)

    mapped_pixels = np.zeros_like(pixels)
    dmc_colors = [entry['rgb_value'] for entry in mapped_palette]
    for idx, color_idx in enumerate(labels):
        mapped_pixels[idx] = dmc_colors[color_idx]

    mapped_image_np = mapped_pixels.reshape((height, width, 3))
    mapped_image = Image.fromarray(mapped_image_np.astype('uint8'), 'RGB')

    if filename == 'output_image.png':
        save_image(mapped_image, '/Users/kateportalatin/stitch_companion_2/images/output/'+filename)    
    else:
        save_image(mapped_image, '/Users/kateportalatin/stitch_companion_2/images/output/'+filename[0]+'_output.png')
    
    return mapped_image


In [46]:
def generate_legend(mapped_palette, symbol_map, filename='legend.png'):
    
    num_colors = len(mapped_palette)
    row_height = 40
    img_width = 400
    img_height = row_height * num_colors
    
    # Create a new image
    legend_img = Image.new('RGB', (img_width, img_height), 'white')
    draw = ImageDraw.Draw(legend_img)
    
    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except IOError:
        # If arial.ttf is not available, use default font
        font = ImageFont.load_default()
    
    for idx, entry in enumerate(mapped_palette):
        y = idx * row_height
        # Draw color square
        color_square_size = 30
        color_square_x = 10
        color_square_y = y + (row_height - color_square_size) // 2
        color = entry['rgb_value']
        draw.rectangle([color_square_x, color_square_y, color_square_x + color_square_size, color_square_y + color_square_size], fill=color, outline='black')
        
        # Get thread_name and symbol
        thread_name = entry['thread_name']
        symbol = symbol_map[tuple(entry['rgb_value'])]
        
        # Draw text: thread_name and symbol
        text_x = color_square_x + color_square_size + 10
        text_y = y + (row_height - 16) // 2  # 16 is approximate font size
        
        draw.text((text_x, text_y), f"{thread_name} ({symbol})", fill='black', font=font)
    
    legend_img.show()
    
    if filename == 'legend.png':
        save_image(legend_img, '/Users/kateportalatin/stitch_companion_2/images/legend/'+filename)
    else:
        new_filename = filename[0] + '_legend.png'
        save_image(legend_img, '/Users/kateportalatin/stitch_companion_2/images/legend/'+new_filename)

In [47]:
key_map, threads = symbol_dict_create(dmc_colors, palette_colors)
symbol_grid = apply_palette_to_image(quantized_image, labels, palette_colors)
legend = generate_legend(palette_colors, key_map)

In [48]:
def generate_pattern_grid(image, symbol_map, GRID_SIZE, filename='pattern_grid.png', max_rows_per_page=75, max_cols_per_page=75):

    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except IOError:
        # If arial.ttf is not available, use default font
        font = ImageFont.load_default()

    # Get image dimensions
    width, height = image.size

    # Calculate the number of cells in x and y directions
    cell_width = GRID_SIZE[0]
    cell_height = GRID_SIZE[1]
    num_cells_x = width // cell_width
    num_cells_y = height // cell_height

    # Create a new image for the pattern grid
    # Each cell can be represented as pix x pix pixels
    pattern_cell_size = 4
    pattern_img_width = num_cells_x * pattern_cell_size
    pattern_img_height = num_cells_y * pattern_cell_size

   # Calculate how many pages are needed
    total_pages_x = math.ceil(num_cells_x / max_cols_per_page)
    total_pages_y = math.ceil(num_cells_y / max_rows_per_page)
    pixels = image.load()

    page_counter = 1

      # Loop through pages
    for page_x in range(total_pages_x):
        for page_y in range(total_pages_y):
            # Calculate the region for this page
            x_start = page_x * max_cols_per_page
            x_end = min((page_x + 1) * max_cols_per_page, num_cells_x)
            y_start = page_y * max_rows_per_page
            y_end = min((page_y + 1) * max_rows_per_page, num_cells_y)

            # Create a new image for this page
            page_img_width = (x_end - x_start) * pattern_cell_size
            page_img_height = (y_end - y_start) * pattern_cell_size
            pattern_img = Image.new('RGB', (page_img_width, page_img_height), 'white')
            draw = ImageDraw.Draw(pattern_img)

            # Loop over the cells within this page
            for i in range(x_start, x_end):
                for j in range(y_start, y_end):
                    # Get the pixels in the cell
                    x0 = i * cell_width
                    y0 = j * cell_height
                    x1 = x0 + cell_width
                    y1 = y0 + cell_height

                    # Collect the colors in the cell
                    cell_colors = []
                    for x in range(x0, min(x1, width)):
                        for y in range(y0, min(y1, height)):
                            cell_colors.append(tuple(pixels[x, y]))  # Ensure tuple format

                    # Get the most common color in the cell
                    most_common_color = Counter(cell_colors).most_common(1)[0][0]

                    # Get the symbol corresponding to the color
                    symbol = symbol_map.get(most_common_color, '?')

                    # Draw the cell
                    grid_x = (i - x_start) * pattern_cell_size
                    grid_y = (j - y_start) * pattern_cell_size
                    # Draw the cell border
                    draw.rectangle([grid_x, grid_y, grid_x + pattern_cell_size, grid_y + pattern_cell_size], outline='black')
                    # Draw the symbol centered in the cell
                    text = symbol
                    # Use draw.textbbox() to get text size
                    bbox = draw.textbbox((0, 0), text, font=font)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    text_x = grid_x + (pattern_cell_size - text_width) / 2
                    text_y = grid_y + (pattern_cell_size - text_height) / 2
                    draw.text((text_x, text_y), text, fill='black', font=font)

            pattern_img.show()
            # Save the page image
            new_filename = f"{filename[:-4]}_page_{page_counter}.png"
            save_image(pattern_img, f'/Users/kateportalatin/py_workspace/stitch_companion_2/images/pattern/{new_filename}')
            page_counter += 1
    

In [60]:
def generate_pattern_grid_mag(image, symbol_map, GRID_SIZE, filename='pattern_grid.png', magnification_factor=4, max_rows_per_page=50, max_cols_per_page=50):
    # Load a font (magnified for better visibility)
    try:
        font = ImageFont.truetype("arial.ttf", 11 * magnification_factor)  # Adjust font size based on magnification
    except IOError:
        # If arial.ttf is not available, use default font
        font = ImageFont.load_default()

    # Get image dimensions
    width, height = image.size

    # Calculate the number of cells in x and y directions
    cell_width = GRID_SIZE[0]
    cell_height = GRID_SIZE[1]
    num_cells_x = width // cell_width
    num_cells_y = height // cell_height

    # Adjust the cell size based on the magnification factor
    pattern_cell_size = 4 * magnification_factor  # Scale the cell size
    pattern_img_width = num_cells_x * pattern_cell_size
    pattern_img_height = num_cells_y * pattern_cell_size

    # Calculate how many pages are needed
    total_pages_x = math.ceil(num_cells_x / max_cols_per_page)
    total_pages_y = math.ceil(num_cells_y / max_rows_per_page)

    pixels = image.load()

    page_counter = 1

    # Loop through pages
    for page_x in range(total_pages_x):
        for page_y in range(total_pages_y):
            # Calculate the region for this page
            x_start = page_x * max_cols_per_page
            x_end = min((page_x + 1) * max_cols_per_page, num_cells_x)
            y_start = page_y * max_rows_per_page
            y_end = min((page_y + 1) * max_rows_per_page, num_cells_y)

            # Create a new image for this page (magnified)
            page_img_width = (x_end - x_start) * pattern_cell_size
            page_img_height = (y_end - y_start) * pattern_cell_size
            pattern_img = Image.new('RGB', (page_img_width, page_img_height), 'white')
            draw = ImageDraw.Draw(pattern_img)

            # Loop over the cells within this page
            for i in range(x_start, x_end):
                for j in range(y_start, y_end):
                    # Get the pixels in the cell
                    x0 = i * cell_width
                    y0 = j * cell_height
                    x1 = x0 + cell_width
                    y1 = y0 + cell_height

                    # Collect the colors in the cell
                    cell_colors = []
                    for x in range(x0, min(x1, width)):
                        for y in range(y0, min(y1, height)):
                            cell_colors.append(tuple(pixels[x, y]))  # Ensure tuple format

                    # Get the most common color in the cell
                    most_common_color = Counter(cell_colors).most_common(1)[0][0]

                    # Get the symbol corresponding to the color
                    symbol = symbol_map.get(most_common_color, '?')

                    # Draw the cell (magnified)
                    grid_x = (i - x_start) * pattern_cell_size
                    grid_y = (j - y_start) * pattern_cell_size
                    # Draw the cell border
                    draw.rectangle([grid_x, grid_y, grid_x + pattern_cell_size, grid_y + pattern_cell_size], outline='black')
                    # Draw the symbol centered in the magnified cell
                    text = symbol
                    # Use draw.textbbox() to get text size
                    bbox = draw.textbbox((0, 0), text, font=font)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    text_x = grid_x + (pattern_cell_size - text_width) / 2
                    text_y = grid_y + (pattern_cell_size - text_height) / 2
                    draw.text((text_x, text_y), text, fill='black', font=font)
            
            # Extract the base name and extension properly
            base_name, ext = os.path.splitext(filename)

            # Create the new filename using the base name and keeping the correct extension
            new_filename = f"{base_name}_magnified_page_{page_counter}{ext}"
            save_image(pattern_img, f'/Users/kateportalatin/py_workspace/stitch_companion_2/images/pattern/{new_filename}')
            page_counter += 1


In [61]:
def detect_bg_color(image, n_clusters=1):
    """
    Detects dominant color in background by sampling edges and uses KMeans clustering
    to find most common color.

    Parameters:
        - image (numpy array): input image (numpy.ndarray)
        - num_clusters (int): Number of clusters used for detection in KMeans clustering

    Returns:
        - numpy array: Detected dominant bg color in HSV format
    """
    print(f"Image shape: {image.shape}")
    
    # check for grayscale
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        
    # check for RGBA 4 channel colors
    if image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)
    
    # get image dimensions 
    height, width, _ = image.shape

    # sample pixels around edges
    top_ = image[0, :, :]
    bottom_ = image[height-1, :, :]
    left_ = image[:, 0, :]
    right_ = image[:, width-1, :]

    horizontals_ = np.vstack([top_, bottom_]).reshape(-1, 3)
    verticals_ = np.vstack([left_, right_]).reshape(-1, 3)
    border_pixels = np.concatenate([horizontals_, verticals_], axis=0)
    reshaped_borders = border_pixels.reshape((-1, 1, 3))
    print(f"Border pixels shape: {border_pixels.shape}")

    # Convert sampled edge pixels to HSV
    hsv_pixels = cv2.cvtColor(reshaped_borders, cv2.COLOR_BGR2HSV)

    # use Kmeans clustering to find dominant bg color
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(hsv_pixels.reshape(-1, 3))
    dominant_hsv = kmeans.cluster_centers_[0].astype(int)

    return dominant_hsv

In [62]:
def color_based_segmentation_bg(image_path, tolerance=-10): 
    image = cv2.imread(image_path)
    dominant_color = detect_bg_color(image)
    
    # Define color range for bg (e.g., detect color)
    lower_bound = np.array([dominant_color[0] - tolerance, 50, 50])
    upper_bound = np.array([dominant_color[0] + tolerance, 255, 255])
    
    # Convert image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # create mask and invert it to isolate bg 
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask_inv = cv2.bitwise_not(mask)

    # apply mask to image
    foreground = cv2.bitwise_and(image, image, mask=mask_inv)
    
    cv2.imshow('Foreground image', foreground)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [63]:
def color_based_segmentation(image, tolerance=-10):
    # Load the PIL image into nimy
    image = np.array(image)

    # Convert RGB to BGR for OpenCV)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Detect the dominant background color
    dominant_color = detect_bg_color(image)
    
    # Define HSV tolerance range for the background color
    lower_bound = np.array([dominant_color[0] - tolerance, 50, 50])
    upper_bound = np.array([dominant_color[0] + tolerance, 255, 255])
    
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Create a mask for the background color
    bg_mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    
    # Invert the mask to get the foreground
    fg_mask = cv2.bitwise_not(bg_mask)
    
    # Clean up the mask using morphological operations (optional)
    kernel = np.ones((5,5), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)  # Remove noise
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel) # Close gaps
    
    # Extract the foreground by applying the mask
    foreground = cv2.bitwise_and(image, image, mask=fg_mask)
    
    # Convert BGR to RGB for PIL
    foreground_rgb = cv2.cvtColor(foreground, cv2.COLOR_BGR2RGB)

    # Convert numpy array to PIL image and return
    return Image.fromarray(foreground_rgb)


In [64]:
image_path = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-1.jpeg"
with Image.open(image_path) as image:
    resized_image = load_and_resize_image(image)
    original_color_count = get_unique_color_count(image)
    print(f"num colors: {original_color_count}")
    num_colors = 31
    ratio = original_color_count / num_colors
    print(f"ratio: {ratio}")
    print(type(resized_image))
    foreground = color_based_segmentation(resized_image)
    
    foreground.show()
    quantized_image, labels, palette = quantize_image(image, num_colors)
    dmc_colors = load_threads('/Users/kateportalatin/py_workspace/stitch_companion_2/dmc_colors.csv')
    palette_colors = map_colors_to_thread(palette, dmc_colors)
    key_map, threads = symbol_dict_create(dmc_colors, palette_colors)
    final_ = apply_palette_to_image(quantized_image,labels, palette_colors, filename='image_output.png')
    generate_legend(palette_colors, key_map)

    # Adjust magnification and specify number of cols and rows per page
    MAG_FACTOR = 2
    NUM_COLS = 150
    NUM_ROWS = 200
    FILENAME = 
    
    generate_pattern_grid_mag(final_, key_map, GRID_SIZE, MAG_FACTOR, NUM_ROWS, NUM_COLS)
    final_.show()

num colors: 39957
ratio: 1288.9354838709678
<class 'PIL.Image.Image'>
Image shape: (200, 157, 3)
Border pixels shape: (714, 3)


TypeError: expected str, bytes or os.PathLike object, not int

In [39]:
# Exploring bg blocking techniques
_img1 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-1.jpeg"
_img2 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-2.jpeg"
_img3 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-3.jpeg"
_img4 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-4.jpeg"
_img5 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-5.jpeg"
_img6 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-6.jpeg"
_img7 = "/Users/kateportalatin/py_workspace/stitch_companion_2/images/sailor-7.png"

MAX_WIDTH = 150
MAX_HEIGHT = 200
GRID_SIZE = (1, 1)
MAX_DIMENSION = minimumOf(MAX_WIDTH, MAX_HEIGHT)

# threshold_for_bg(_img1)    
# grabcut_for_bg(_img1)
color_based_segmentation_bg(_img1)
color_based_segmentation_bg(_img2)
color_based_segmentation_bg(_img3)
color_based_segmentation_bg(_img4)
color_based_segmentation_bg(_img5)
color_based_segmentation_bg(_img6)

# threshold_for_bg(_img3)    
# grabcut_for_bg(_img3)
# color_based_segmentation_bg(_img3)

with Image.open(_img4) as image:
    print("for image 4:")
    color_count = get_unique_color_count(image)    
    width, height = image.size
    print(f"Size: {width} x {height}\n")
    
with Image.open(_img1) as image:
    print("for image 1:")
    color_count = get_unique_color_count(image)    
    width, height = image.size
    print(f"Size: {width} x {height}\n")
    
    # resized_image = load_and_resize_image(image)
    # resized_width, resized_height, resized_pixels = get_number_of_pixels(resized_image)
    # print(f"width: {resized_width}, height: {resized_height}, total_ = {resized_pixels}")

    # dmc_colors = load_threads('/Users/kateportalatin/py_workspace/stitch_companion_2/dmc_colors.csv')
    # quantized_image, labels, palette = quantize_image(image, palette_size)
    # palette_colors = map_colors_to_thread(palette, dmc_colors)

    # key_map, threads = symbol_dict_create(dmc_colors, palette_colors)
    # final_ = apply_palette_to_image(quantized_image, labels, palette_colors, filename=filename)
    # legend = generate_legend(palette_colors, key_map, filename=filename)
    # generate_pattern_grid(final_, key_map, GRID_SIZE, filename=filename)

    # final_.show()
        

Image shape: (319, 251, 3)
Border pixels shape: (1140, 3)
Image shape: (225, 225, 3)
Border pixels shape: (900, 3)
Image shape: (259, 194, 3)
Border pixels shape: (906, 3)
Image shape: (189, 267, 3)
Border pixels shape: (912, 3)
Image shape: (225, 225, 3)
Border pixels shape: (900, 3)
Image shape: (259, 194, 3)
Border pixels shape: (906, 3)
for image 4:
Size: 267 x 189

for image 1:
Size: 251 x 319

